In [1]:
pip install SQLAlchemy==1.4.17

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for SQLAlchemy: filename=SQLAlchemy-1.4.17-cp310-cp310-linux_x86_64.whl size=1519925 sha256=48aa8fc9ae7684267aa426b9a55df4dd1ed36f5581de35ec31edbb83827adffc
  Stored in directory: /root/.cache/pip/wheels/eb/d0/1e/ab0fcbac6f74a69b788a37fccd365fc67535b5ab79f97bd860
Successfully built SQLAlchemy
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.29
    Uninstalling SQLAlchemy-2.0.29:
      Successfully uninstalled SQLAlchemy-2.0.29
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.17 which is incompatible.


In [2]:
pip install pymssql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 15.4 MB/s eta 0:00:00


In [3]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.8 MB/s eta 0:00:00


In [4]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
import statsmodels.formula.api as smf
from pandas_datareader import DataReader as pdr

from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor as rf
from catboost import CatBoostRegressor as sbr

In [5]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912"
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

## Strategy 1

In [6]:
df = pd.read_sql(
    """
    select date, ticker, mom1m, mom12m, mom36m, siccd, ret, sfe, rsup, ear, chmom, idiovol
    from data
    order by date, ticker
    """,
    conn
)
df = df.dropna()
df = df.set_index(["date", "ticker"])
features = ["mom1m", "sfe", "rsup", "ear", "chmom", "idiovol"]

In [7]:
df["actual"] = df.ret

qt = QuantileTransformer(output_distribution="normal")

grouped = df.groupby("date", group_keys=False)
df[features+["ret"]] = grouped[features+["ret"]].apply(
  lambda d:
    pd.DataFrame(
      qt.fit_transform(d),
      columns=d.columns,
      index=d.index
    )
)

In [8]:
inds = pd.read_csv("siccodes12.csv", index_col="industry")
ind_names = inds.index.unique().to_list()

def industry(sic):
  try:
    return inds[(inds.start<=sic)&(sic<=inds.end)].index[0]
  except:
    return "Other"

codes = pd.Series({code: industry(code) for code in df.siccd.unique()})
codes = pd.DataFrame(codes).reset_index()
codes.columns = ["siccd", "industry"]

df = df.reset_index().merge(codes, on="siccd")
df = df.set_index(["date", "ticker"])

features.append("industry")

Pipeline 1: Also favourite pipeline

quantile transform + column transform + polynomial transform + model

In [9]:
pipe1 = sbr(cat_features=["industry"], random_state=42)


Looping test

In [10]:
dates = ["2005-01", "2010-01", "2015-01", "2020-01", "3000-01"]
predictions = None
train_score = []
test_score = []

for train_date, end_date in zip(dates[:-1], dates[1:]):

  fltr1 = df.index.get_level_values("date") < train_date
  fltr2 = df.index.get_level_values("date") < end_date
  train = df[fltr1]
  test = df[~fltr1 & fltr2]

  Xtrain = train[features]
  ytrain = train["ret"]
  Xtest = test[features]
  ytest = test["ret"]

  pipe1.fit(Xtrain, ytrain)
  print('Train set score: ' + str(pipe1.score(Xtrain, ytrain)))


  pred = pipe1.predict(Xtest)
  pred = pd.Series(pred, index=test.index)
  predictions = pd.concat((predictions, pred))
  print('Test set score: ' + str(pipe1.score(Xtest,ytest)))

df["predict"] = predictions

Learning rate set to 0.091534
0:	learn: 1.0061573	total: 142ms	remaining: 2m 21s
1:	learn: 1.0057260	total: 200ms	remaining: 1m 39s
2:	learn: 1.0053243	total: 255ms	remaining: 1m 24s
3:	learn: 1.0049123	total: 306ms	remaining: 1m 16s
4:	learn: 1.0045713	total: 362ms	remaining: 1m 11s
5:	learn: 1.0042714	total: 414ms	remaining: 1m 8s
6:	learn: 1.0040893	total: 467ms	remaining: 1m 6s
7:	learn: 1.0038481	total: 530ms	remaining: 1m 5s
8:	learn: 1.0036132	total: 591ms	remaining: 1m 5s
9:	learn: 1.0034612	total: 649ms	remaining: 1m 4s
10:	learn: 1.0032719	total: 701ms	remaining: 1m 3s
11:	learn: 1.0030945	total: 755ms	remaining: 1m 2s
12:	learn: 1.0029759	total: 808ms	remaining: 1m 1s
13:	learn: 1.0028622	total: 858ms	remaining: 1m
14:	learn: 1.0027084	total: 916ms	remaining: 1m
15:	learn: 1.0026034	total: 984ms	remaining: 1m
16:	learn: 1.0024945	total: 1.05s	remaining: 1m
17:	learn: 1.0023352	total: 1.1s	remaining: 1m
18:	learn: 1.0022195	total: 1.16s	remaining: 59.8s
19:	learn: 1.0020976	t

In [11]:
df

mom1m    mom12m    mom36m  siccd       ret       sfe  \
date    ticker                                                            
2000-01 AA      1.174124  0.783092  0.207338   3334 -1.067024  0.465527   
        KLU     0.834544  0.358974 -0.580645   3334 -1.552878 -1.531518   
2000-02 AA     -1.056102  1.016135  0.250118   3334 -0.024881  1.103803   
        KLU    -1.544094  0.500000 -0.601942   3334 -0.315010 -1.529685   
2000-03 AA     -0.033619  0.737681  0.175634   3334 -0.026778  0.869391   
...                  ...       ...       ...    ...       ...       ...   
2021-11 KSPN   -1.634799  0.481778 -0.426020   5735 -0.316798 -1.154158   
2021-12 KSPN   -0.317081  0.135848 -0.346592   5735 -1.866321 -1.149192   
2022-01 KSPN   -1.866837 -0.070403  0.161111   5735 -1.916238 -1.185322   
2022-02 KSPN   -1.898205 -0.729253  2.295728   5735  0.556408 -1.166941   
2022-03 KSPN    0.552446 -0.764890  2.393618   5735 -0.834250 -1.117105   

                    rsup       ear     chmom   idiovol    actual  \
date    ticker                                                     
2000-01 AA      0.261592 -0.063497 -0.619031 -0.438334 -0.160392   
        KLU    -1.692193 -1.158489 -1.731264  0.373475 -0.219512   
2000-02 AA      0.247164 -0.037780 -0.486276 -0.451970 -0.013453   
        KLU    -1.703658 -1.149544 -1.951912  0.373893 -0.052083   
2000-03 AA     -0.711993 -0.307048 -1.138947 -0.444358  0.025547   
...                  ...       ...       ...       ...       ...   
2021-11 KSPN    1.813806 -0.284599 -1.322762  1.962924 -0.062392   
2021-12 KSPN    1.443499 -0.436823 -1.613696  1.951665 -0.232353   
2022-01 KSPN   -2.378046 -0.949205 -1.652350  1.966406 -0.281609   
2022-02 KSPN   -2.330739 -0.930033  0.152451  1.994387  0.047333   
2022-03 KSPN   -2.203424 -0.821604 -0.550940  2.013429 -0.075646   

                     industry   predict  
date    ticker                           
2000-01 AA      Manufacturing       NaN  
        KLU     Manufacturing       NaN  
2000-02 AA      Manufacturing       NaN  
        KLU     Manufacturing       NaN  
2000-03 AA      Manufacturing       NaN  
...                       ...       ...  
2021-11 KSPN            Shops -0.075260  
2021-12 KSPN            Shops -0.053228  
2022-01 KSPN            Shops  0.038822  
2022-02 KSPN            Shops -0.157702  
2022-03 KSPN            Shops  0.045661  

[670388 rows x 13 columns]

In [12]:
df.to_csv('strat1.csv')

In [13]:
numlong = 150
numshort = 100

df["rank_from_top"] = df.groupby("date").predict.rank(
  method="first",
  ascending=False
)
df["long"] = df.rank_from_top <= numlong

df["rank_from_bottom"] = df.groupby("date").predict.rank(
  method="first"
)
df["short"] = df.rank_from_bottom <= numshort

In [14]:
dates = ["2005-01", "2010-01", "2015-01", "2020-01", "3000-01"]
df = df[df.index.get_level_values("date") >= dates[0]]

long_ret = df[df.long].groupby("date").actual.mean()
short_ret = df[df.short].groupby("date").actual.mean()

In [15]:
# 150.50 return
ret = 1.5*long_ret - 0.5*short_ret
ret

date
2005-01    0.005693
2005-02    0.014410
2005-03   -0.016364
2005-04   -0.077529
2005-05    0.113878
             ...   
2021-11   -0.009839
2021-12    0.101805
2022-01   -0.042940
2022-02    0.021738
2022-03   -0.024967
Name: actual, Length: 207, dtype: float64

In [16]:
df_new = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=2005)[0]/100
df_new.index = df_new.index.astype(str)
df_new["ret"] = ret
df_new["ret_rf"] = df_new.ret - df_new.RF
df_new = df_new.dropna()
df_new.index = df_new.index.astype(str)
df_new.index.name = "date"
df_new = df_new.reset_index()
df_new = df_new.rename(columns={"Mkt-RF": "mkt_rf", "RF": "rf"})

<ipython-input-16-c9ad2a6d9b27>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_new = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=2005)[0]/100
<ipython-input-16-c9ad2a6d9b27>:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_new = pdr("F-F_Research_Data_5_Factors_2x3", "famafrench", start=2005)[0]/100


In [17]:
df_new

,date,mkt_rf,SMB,HML,RMW,CMA,rf,ret,ret_rf
0,2005-01,-0.0276,-0.0117,0.0206,0.0274,-0.0146,0.0016,0.005693,0.004093
1,2005-02,0.0189,-0.0031,0.0154,0.0143,-0.0005,0.0016,0.014410,0.012810
2,2005-03,-0.0197,-0.0142,0.0204,0.0047,0.0129,0.0021,-0.016364,-0.018464
3,2005-04,-0.0261,-0.0398,0.0007,0.0097,-0.0094,0.0021,-0.077529,-0.079629
4,2005-05,0.0365,0.0279,-0.0064,-0.0100,0.0030,0.0024,0.113878,0.111478
...,...,...,...,...,...,...,...,...,...
202,2021-11,-0.0155,-0.0177,-0.0045,0.0720,0.0173,0.0000,-0.009839,-0.009839
203,2021-12,0.0310,-0.0080,0.0326,0.0491,0.0440,0.0001,0.101805,0.101705
204,2022-01,-0.0625,-0.0406,0.1275,0.0084,0.0772,0.0000,-0.042940,-0.042940
205,2022-02,-0.0229,0.0292,0.0308,-0.0209,0.0312,0.0000,0.021738,0.021738


In [18]:
#Computing alpha
result = smf.ols("ret_rf~mkt_rf+SMB+HML+CMA+RMW", df_new).fit()
betas = result.params[1:]
mkt = df_new.rf + betas[0]*df_new.mkt_rf
smb = betas[1]*df_new.SMB
hml = betas[2]*df_new.HML
cma = betas[3]*df_new.CMA
rmw = betas[4]*df_new.RMW
active = df_new.ret - mkt - smb - hml - cma - rmw


In [19]:
print("Active is found to be", active)

Active is found to be 0      0.032260
1     -0.013472
2      0.004354
3     -0.031239
4      0.058533
         ...   
202    0.021499
203    0.062144
204    0.005623
205    0.024350
206   -0.039353
Length: 207, dtype: float64


In [20]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 ret_rf   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.730
Method:                 Least Squares   F-statistic:                     112.3
Date:                Thu, 02 May 2024   Prob (F-statistic):           3.04e-56
Time:                        22:23:30   Log-Likelihood:                 392.88
No. Observations:                 207   AIC:                            -773.8
Df Residuals:                     201   BIC:                            -753.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0095      0.003      3.541      0.0